In [9]:
%matplotlib inline

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

In [11]:
# LOADS CIFAR10 images which are 32 x 32 x 3 RGB images
# iter(trainloader/testloader) are iterables that come in pairs (image, label)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Choose Hyperparameters

In [12]:
num_epochs = 12 # number of epochs to train for
momentum = 0.9 # momentum for Stochastic Gradient Descent
lr = 0.001 # learning rate (eta) for gradient descent
M = 100  # number of neurons in hidden layer of neural network

# Build Neural Network

In [13]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, M):
        super(Net, self).__init__()
        
        # in features --> flattened 32 x 32 x 3 image, out features = label (0,1,..., 9), use bias 
        self.linear1 = nn.Linear(3072, M, bias=True)
        self.linear2 = nn.Linear(M, 10, bias=True)

    def forward(self, x):
        x = x.view(4, -1)
        x = self.linear1(x)
        x = F.relu(x)
        return self.linear2(x)


net = Net(M)

# 3. Define a Loss function and optimizer

Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

# 4. Train the network


This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



In [15]:
def calc_accuracy(dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return 100.0 * correct / total

In [16]:
all_train_accuracies = []
all_test_accuracies = []
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
    train_accuracy = calc_accuracy(trainloader)
    test_accuracy = calc_accuracy(testloader)
    print('END OF EPOCH ', epoch + 1, ': train accuracy = ', train_accuracy )
    
    all_train_accuracies.append(train_accuracy)
    all_test_accuracies.append(test_accuracy)

print('Finished Training')

[1,  2000] loss: 1.847
[1,  4000] loss: 1.712


KeyboardInterrupt: 

# Plot accuracy over time






In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
plt.plot(all_train_accuracies)
plt.plot(all_test_accuracies)
plt.title('Training and Testing Accuracy after each iteration')
plt.xlabel('Iteration Number')
plt.ylabel('Accuracy (%)')
plt.legend(['Training', 'Testing'])
plt.show()